In [16]:
import numpy as np
import random
import os
import re

In [4]:
import tensorflow as tf
print tf.__file__

/Users/jorge/anaconda/lib/python2.7/site-packages/tensorflow/__init__.pyc


In [6]:
#Use a pretrained model
path_model = '/Users/jorge/tensorflow/tensorflow/models/image/imagenet'


#Create graph
with tf.gfile.FastGFile(os.path.join(path_model, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')


In [21]:
print graph_def.ListFields

In [9]:
#Image data
image = os.path.join(path_model, 'cropped_panda.jpg')
image_data = tf.gfile.FastGFile(image, 'rb').read()


In [10]:
#Generate predictios from the jpg file
with tf.Session() as sess:
    softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
    predictions = sess.run(softmax_tensor,
                           {'DecodeJpeg/contents:0': image_data})
predictions = np.squeeze(predictions)

print predictions

[[  1.00235826e-04   2.51057325e-04   7.98699330e-05 ...,   1.00235244e-04
    1.00236299e-04   1.00236299e-04]]


In [17]:
class NodeLookup(object):
  """Converts integer node ID's to human readable labels."""

  def __init__(self,
               label_lookup_path=None,
               uid_lookup_path=None):
    if not label_lookup_path:
      label_lookup_path = os.path.join(
          path_model, 'imagenet_2012_challenge_label_map_proto.pbtxt')
    if not uid_lookup_path:
      uid_lookup_path = os.path.join(
          path_model, 'imagenet_synset_to_human_label_map.txt')
    self.node_lookup = self.load(label_lookup_path, uid_lookup_path)

    
  def load(self, label_lookup_path, uid_lookup_path):
    """Loads a human readable English name for each softmax node.

    Args:
      label_lookup_path: string UID to integer node ID.
      uid_lookup_path: string UID to human-readable string.

    Returns:
      dict from integer node ID to human-readable string.
    """
    if not tf.gfile.Exists(uid_lookup_path):
      tf.logging.fatal('File does not exist %s', uid_lookup_path)
    if not tf.gfile.Exists(label_lookup_path):
      tf.logging.fatal('File does not exist %s', label_lookup_path)

    # Loads mapping from string UID to human-readable string
    proto_as_ascii_lines = tf.gfile.GFile(uid_lookup_path).readlines()
    uid_to_human = {}
    p = re.compile(r'[n\d]*[ \S,]*')
    for line in proto_as_ascii_lines:
      parsed_items = p.findall(line)
      uid = parsed_items[0]
      human_string = parsed_items[2]
      uid_to_human[uid] = human_string

    # Loads mapping from string UID to integer node ID.
    node_id_to_uid = {}
    proto_as_ascii = tf.gfile.GFile(label_lookup_path).readlines()
    for line in proto_as_ascii:
      if line.startswith('  target_class:'):
        target_class = int(line.split(': ')[1])
      if line.startswith('  target_class_string:'):
        target_class_string = line.split(': ')[1]
        node_id_to_uid[target_class] = target_class_string[1:-2]

    # Loads the final mapping of integer node ID to human-readable string
    node_id_to_name = {}
    for key, val in node_id_to_uid.items():
      if val not in uid_to_human:
        tf.logging.fatal('Failed to locate: %s', val)
      name = uid_to_human[val]
      node_id_to_name[key] = name

    return node_id_to_name


  def id_to_string(self, node_id):
    if node_id not in self.node_lookup:
      return ''
    return self.node_lookup[node_id]


In [18]:
# Creates node ID --> English string lookup.
node_lookup = NodeLookup()

top_k = predictions.argsort()[-5:][::-1]
for node_id in top_k:
    human_string = node_lookup.id_to_string(node_id)
    score = predictions[node_id]
    print('%s (score = %.5f)' % (human_string, score))

giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca (score = 0.89233)
indri, indris, Indri indri, Indri brevicaudatus (score = 0.00859)
lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens (score = 0.00264)
custard apple (score = 0.00141)
earthstar (score = 0.00107)


In [9]:
print os.listdir('/Users/jorge')

['.bash_history', '.bash_profile', '.bash_profile-anaconda.bak', '.bash_sessions', '.cache', '.CFUserTextEncoding', '.cmake', '.config', '.continuum', '.cups', '.designer', '.dropbox', '.DS_Store', '.gitconfig', '.ipython', '.jupyter', '.local', '.matplotlib', '.mono', '.oracle_jre_usage', '.profile', '.putty', '.PyCharm50', '.R', '.Rapp.history', '.recently-used', '.Rhistory', '.rnd', '.rstudio-desktop', '.spyder2', '.ssh', '.subversion', '.tblive-4', '.theano', '.theanorc', '.Trash', '.viminfo', '.wget-hsts', '.Xauthority', 'Admin', 'aeat', 'anaconda', 'Applications', 'Biblioteca de calibre', 'caffe', 'Desktop', 'Documents', 'Downloads', 'Dropbox', 'dumps', 'envs', 'Library', 'Movies', 'Music', 'nvidia', 'NVIDIA_CUDA-7.5_Samples', 'oneDrive', 'Pictures', 'projects', 'Public', 'PycharmProjects', 'Reference', 'tensorflow', 'torch', 'VirtualBox VMs', 'working']


In [13]:
print ord('1')
print chr(45)

49
-


In [ ]:
def create_word(word_char, im_path='/path to curated univenda chars/'):
    word_img = np.zeros((64:64*len(word_char)))
    character_cuts = []
    
    pos_fin_prev = 0
    for c in word_char:
        #Select random char image from the dir of specific character
        img_list = os.listdir(im_path + ord(c))
        n_img = int(random.uniform(0,len(img_list)))
        img_c = plt.imread(im_path + str(ord(c)) + '/' + img_list[n_img])

        #Calculate the initial and final positions of the character line by line
        pos_ini = np.zeros(img_c.shape[1])
        pos_fin = np.zeros(img_c.shape[1])
        for row, line in enumerate(img_c):
            init_found = False
            for column, value in enumerate line:
                if not(init_found) and value>0:
                    init_found = True
                    pos_ini[row] = column
                if init_found and value>0:
                    pos_fin[row] = column
        min_ini = pos_ini.min()
        max_fin = pos_fin.max()
        len_char = max_fin - min_ini
        
        #put the new char over the image
        word_img[:,pos_fin_prev:pos_fin_prev + len_char] = max( word_img[:,pos_fin_prev:pos_fin_prev + len_char],
                                                               img_c[:,min_ini:max_fin])
        pos_fin_prev = pos_fin_prev + len_char
        character_cuts += [pos_fin_prev + len_char]
        
        
    return word_img, character_cuts



